In [23]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [25]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term


In [26]:
import pandas as pd

from virny.datasets.data_loaders import ACSIncomeDataset
from virny.utils.protected_groups_partitioning import create_test_protected_groups

In [57]:
def get_proportions(protected_groups, X_data):
    for col_name in protected_groups.keys():
        proportion = protected_groups[col_name].shape[0] / X_data.shape[0]
        print(f'{col_name}: {round(proportion, 3)}')


def get_base_rate(protected_groups, y_data):
    for col_name in protected_groups.keys():
        filtered_df = y_data.iloc[protected_groups[col_name].index].copy(deep=True)
        base_rate = filtered_df[filtered_df == 1].shape[0] / filtered_df.shape[0]
        print(f'{col_name}: {round(base_rate, 3)}')

    base_rate = y_data[y_data == 1].shape[0] / y_data.shape[0]
    print(f'overall: {round(base_rate, 3)}')

## ACSIncome

In [28]:
sensitive_attributes_dct = {'SEX': '2', 'RAC1P': ['2', '3', '4', '5', '6', '7', '8', '9'], 'SEX & RAC1P': None}

In [29]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False, subsample_size=15_000, subsample_seed=42)
data_loader.full_df.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [30]:
data_loader.full_df.shape

(15000, 10)

In [31]:
data_loader.full_df['SEX'].value_counts()

1    7669
2    7331
Name: SEX, dtype: int64

In [32]:
protected_groups = create_test_protected_groups(data_loader.X_data, data_loader.X_data, sensitive_attributes_dct)

In [33]:
for col_name in protected_groups.keys():
    print(f'{col_name}: {protected_groups[col_name].shape[0]}')

SEX_priv: 7669
SEX_dis: 7331
RAC1P_priv: 10163
RAC1P_dis: 4837
SEX&RAC1P_priv: 12431
SEX&RAC1P_dis: 2569


In [34]:
protected_groups.keys()

dict_keys(['SEX_priv', 'SEX_dis', 'RAC1P_priv', 'RAC1P_dis', 'SEX&RAC1P_priv', 'SEX&RAC1P_dis'])

In [52]:
get_proportions(protected_groups, data_loader.X_data)

SEX_priv: 0.511
SEX_dis: 0.489
RAC1P_priv: 0.678
RAC1P_dis: 0.322
SEX&RAC1P_priv: 0.829
SEX&RAC1P_dis: 0.171


In [58]:
get_base_rate(protected_groups, data_loader.y_data)

SEX_priv: 0.422
SEX_dis: 0.274
RAC1P_priv: 0.389
RAC1P_dis: 0.267
SEX&RAC1P_priv: 0.374
SEX&RAC1P_dis: 0.232
overall: 0.35


In [37]:
data_loader.y_data.iloc[protected_groups['SEX_priv'].index].head()

0    1
3    0
5    1
6    1
7    0
Name: PINCP, dtype: int64

In [38]:
data_loader.y_data.iloc[protected_groups['SEX_priv'].index].dtype

dtype('int64')

In [39]:
filtered_df = data_loader.y_data.iloc[protected_groups['SEX_priv'].index].copy(deep=True)
base_rate = filtered_df[filtered_df == 1].shape[0] / filtered_df.shape[0]

In [40]:
base_rate

0.4219585343591081

In [41]:
filtered_df[filtered_df == 1].shape[0]

3236

In [42]:
filtered_df.shape[0]

7669